## try n times decorator

In [39]:
import functools
import time
from loguru import logger
from typing import Callable, Any, Optional

class MaxRetriesExceeded(Exception):
    pass

# request
def try_n_times(
    max_retries: int = 3, delay = 1, 
):
    """
    Try to run a func for n times with pauses in between runs
    """

    def decorate(func):
        @functools.wraps(func)
        def new_func(*args, **kwargs):
            for _ in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if _ == max_retries - 1: # terminal case
                        logger.warning(f"{func.__name__} failed with error: {e}, reached max retries")
                    else:
                        # processing logic
                        logger.warning(f"{func.__name__} failed with error: {e}, will retry in {delay} seconds...")
                        time.sleep(delay)
            raise MaxRetriesExceeded(
                            f"Max retries {max_retries} exceeded for {func.__name__}"
                        )
        return new_func

    return decorate



In [40]:
@try_n_times(max_retries=3, delay=2)
def unreliable_function(x):
    """
    A function that raises an exception unless x > 5.
    """
    if x <= 5:
        raise ValueError("x must be greater than 5")
    return f"Success with x = {x}"


In [41]:

# Test case 2: Function fails and returns fallback value
try:
    result = unreliable_function(4)
except MaxRetriesExceeded as e:
    logger.error(e)

2025-04-23 00:17:04.296 | WARNING  | __main__:new_func:28 - unreliable_function failed with error: x must be greater than 5, will retry in 2 seconds...
2025-04-23 00:17:06.297 | WARNING  | __main__:new_func:28 - unreliable_function failed with error: x must be greater than 5, will retry in 2 seconds...
2025-04-23 00:17:08.300 | WARNING  | __main__:new_func:25 - unreliable_function failed with error: x must be greater than 5, reached max retries
2025-04-23 00:17:08.303 | ERROR    | __main__:<module>:5 - Max retries 3 exceeded for unreliable_function
